# EfficientNetB2 - Waste Classification
## TensorFlow Implementation (Windows/CUDA)

**Model:** EfficientNetB2 (ImageNet pretrained)

**Objective:** Compare performance between preprocessed and raw datasets

**Hardware:** Windows RTX 3060 Ti (CUDA)

**Classes:** aluminium, paper, plastic

---
## 1. Import Libraries and Setup

In [ ]:
# Install TensorFlow 2.20.0 GPU version for Windows CUDA 12.7
# TensorFlow 2.20.0 supports CUDA 12.7
!pip uninstall -y tensorflow
!pip install tensorflow[and-cuda]==2.20.0

# Alternative manual installation if GPU doesn't work:
# !pip install tensorflow==2.20.0
# !pip install nvidia-cuda-runtime-cu12 nvidia-cudnn-cu12 nvidia-cublas-cu12

  Using cached tensorflow-2.20.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.5 kB)
  Using cached protobuf-6.33.5-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached termcolor-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached ml_dtypes-0.5.4-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.9 kB)
  Using cached nvidia_cublas_cu12-12.9.1.4-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.9.79-py3-none-manylinux_2_25_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cuda_nvcc_cu12-12.9.86-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.9.86-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.9.79-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  U

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support

# Import custom logger
sys.path.append('..')
from result_logger import log_result

# Check GPU availability and CUDA compatibility
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("CUDA Available:", tf.test.is_built_with_cuda())

# Additional CUDA check
if tf.config.list_physical_devices('GPU'):
    print("✅ GPU is available and TensorFlow can use it!")
    # Test GPU computation
    with tf.device('/GPU:0'):
        a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
        b = tf.constant([[1.0, 0.0], [0.0, 1.0]])
        c = tf.matmul(a, b)
        print("GPU test successful:", c.numpy())
else:
    print("❌ GPU not available. Check CUDA installation.")
    print("💡 Windows Troubleshooting:")
    print("   1. Install CUDA 12.0+ toolkit from NVIDIA website")
    print("   2. Install cuDNN 8.9+ compatible with CUDA 12.x")
    print("   3. Add CUDA bin path to Windows PATH")
    print("   4. Restart VS Code/Jupyter after installation")
    print("   5. Try: pip install nvidia-cuda-runtime-cu12 nvidia-cudnn-cu12")

2026-02-02 02:37:18.897183: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-02 02:37:20.118448: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-02 02:37:26.907487: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.16.1
GPU Available: []
CUDA Available: True
❌ GPU not available. Check CUDA installation.
💡 Windows Troubleshooting:
   1. Install CUDA 12.0+ toolkit from NVIDIA website
   2. Install cuDNN 8.9+ compatible with CUDA 12.x
   3. Add CUDA bin path to Windows PATH
   4. Restart VS Code/Jupyter after installation
   5. Try: pip install nvidia-cuda-runtime-cu12 nvidia-cudnn-cu12


2026-02-02 02:37:39.876039: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2026-02-02 02:37:39.932488: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


---
## 2. Dataset Paths Configuration
Define all dataset paths as constants

In [ ]:
# Dataset Configuration
CLASSES = ['aluminium', 'paper', 'plastic']
NUM_CLASSES = 3
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Set A: Preprocessed Public (Training Data)
PREPROCESSED_TRAIN = r"../Dataset/preprocessed_Public/train"
PREPROCESSED_VAL = r"../Dataset/preprocessed_Public/val"
PREPROCESSED_TEST_PUBLIC = r"../Dataset/preprocessed_Public/test"

# Set B: Preprocessed Self-Collected (Final Testing)
PREPROCESSED_TEST_SELF = r"../Dataset/preprocessed_self/test"

# Set C: Raw Data (For Comparison)
RAW_PUBLIC = r"../Dataset/Public_dataset"
RAW_SELF = r"../Dataset/SelfCollected_Dataset"

# Create results directory
os.makedirs('results', exist_ok=True)
os.makedirs('models', exist_ok=True)

print("✓ Paths configured successfully")

---
## 3. Data Generators Setup
Create data loaders with proper preprocessing

In [ ]:
def create_data_generators(train_path, val_path, test_path, use_augmentation=True):
    """
    Create data generators for training, validation, and testing
    
    Args:
        train_path: Path to training data
        val_path: Path to validation data (if None, use train_path with validation_split)
        test_path: Path to test data
        use_augmentation: Whether to apply data augmentation
    
    Returns:
        train_gen, val_gen, test_gen
    """
    
    if use_augmentation:
        # Training data generator with augmentation
        train_datagen = ImageDataGenerator(
            preprocessing_function=preprocess_input,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            zoom_range=0.2,
            fill_mode='nearest'
        )
    else:
        train_datagen = ImageDataGenerator(
            preprocessing_function=preprocess_input
        )
    
    # Validation and test generators (no augmentation)
    val_test_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input
    )
    
    # Create generators
    train_gen = train_datagen.flow_from_directory(
        train_path,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True,
        classes=CLASSES
    )
    
    if val_path and os.path.exists(val_path):
        val_gen = val_test_datagen.flow_from_directory(
            val_path,
            target_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='categorical',
            shuffle=False,
            classes=CLASSES
        )
    else:
        # Use validation split from training data
        val_gen = None
    
    test_gen = val_test_datagen.flow_from_directory(
        test_path,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False,
        classes=CLASSES
    )
    
    return train_gen, val_gen, test_gen

print("✓ Data generator function defined")

---
## 4. Model Architecture
Build EfficientNetB2 with custom classification head

In [ ]:
def build_efficientnetb2_model():
    """
    Build EfficientNetB2 model with custom classification head
    
    Returns:
        Compiled Keras model
    """
    
    # Load pretrained EfficientNetB2 (without top classification layer)
    base_model = EfficientNetB2(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
    )
    
    # Freeze base model layers initially
    base_model.trainable = False
    
    # Build custom classification head
    inputs = keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs)
    
    # Compile model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

print("✓ Model architecture function defined")

---
## 5. Training Function
Complete training pipeline with callbacks

In [ ]:
def train_model(model, train_gen, val_gen, model_name, epochs=30, fine_tune=True):
    """
    Train the model with optional fine-tuning
    
    Args:
        model: Keras model
        train_gen: Training data generator
        val_gen: Validation data generator
        model_name: Name for saving the model
        epochs: Number of training epochs
        fine_tune: Whether to fine-tune base model layers
    
    Returns:
        Training history
    """
    
    # Callbacks
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            f'models/{model_name}.keras',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]
    
    print(f"\n{'='*60}")
    print(f"Training Phase 1: Training Classification Head")
    print(f"{'='*60}\n")
    
    # Phase 1: Train with frozen base
    history_phase1 = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs // 2,
        callbacks=callbacks,
        verbose=1
    )
    
    if fine_tune:
        print(f"\n{'='*60}")
        print(f"Training Phase 2: Fine-Tuning Base Model")
        print(f"{'='*60}\n")
        
        # Unfreeze base model for fine-tuning
        model.layers[1].trainable = True  # base_model
        
        # Recompile with lower learning rate
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=1e-5),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # Phase 2: Fine-tune
        history_phase2 = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=epochs // 2,
            callbacks=callbacks,
            verbose=1
        )
        
        # Combine histories
        history = history_phase1
        for key in history_phase1.history.keys():
            history.history[key].extend(history_phase2.history[key])
    else:
        history = history_phase1
    
    return history

print("✓ Training function defined")

---
## 6. Evaluation Function
Evaluate model and generate metrics

In [ ]:
def evaluate_model(model, test_gen, experiment_name):
    """
    Evaluate model and display results
    
    Args:
        model: Trained Keras model
        test_gen: Test data generator
        experiment_name: Name of the experiment
    
    Returns:
        Dictionary with evaluation metrics
    """
    
    print(f"\n{'='*60}")
    print(f"Evaluating: {experiment_name}")
    print(f"{'='*60}\n")
    
    # Get predictions
    test_gen.reset()
    y_pred_probs = model.predict(test_gen, verbose=1)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = test_gen.classes
    
    # Calculate metrics
    test_loss, test_acc = model.evaluate(test_gen, verbose=0)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='weighted'
    )
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=CLASSES))
    
    # Display confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=CLASSES, yticklabels=CLASSES)
    plt.title(f'Confusion Matrix - {experiment_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"Results Summary - {experiment_name}")
    print(f"{'='*60}")
    print(f"Test Loss:      {test_loss:.4f}")
    print(f"Test Accuracy:  {test_acc:.4f}")
    print(f"Precision:      {precision:.4f}")
    print(f"Recall:         {recall:.4f}")
    print(f"F1-Score:       {f1:.4f}")
    print(f"{'='*60}\n")
    
    return {
        'loss': test_loss,
        'accuracy': test_acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

print("✓ Evaluation function defined")

---
## 7. Experiment 1: Preprocessed Dataset
Train on preprocessed public data, test on preprocessed self-collected data

In [ ]:
print("\n" + "#"*60)
print("# EXPERIMENT 1: PREPROCESSED DATASET")
print("#"*60 + "\n")

# Create data generators
train_gen_prep, val_gen_prep, test_gen_prep = create_data_generators(
    PREPROCESSED_TRAIN,
    PREPROCESSED_VAL,
    PREPROCESSED_TEST_SELF,
    use_augmentation=True
)

# Build model
model_prep = build_efficientnetb2_model()
print("\nModel Summary:")
model_prep.summary()

# Train model
history_prep = train_model(
    model_prep,
    train_gen_prep,
    val_gen_prep,
    model_name='efficientnetb2_preprocessed',
    epochs=30,
    fine_tune=True
)

# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
axes[0].plot(history_prep.history['accuracy'], label='Train Accuracy')
axes[0].plot(history_prep.history['val_accuracy'], label='Val Accuracy')
axes[0].set_title('Model Accuracy - Preprocessed')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

# Loss
axes[1].plot(history_prep.history['loss'], label='Train Loss')
axes[1].plot(history_prep.history['val_loss'], label='Val Loss')
axes[1].set_title('Model Loss - Preprocessed')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Evaluate on test set
results_prep = evaluate_model(model_prep, test_gen_prep, "EfficientNetB2 - Preprocessed")

# Log results
log_result(
    model_name='EfficientNetB2',
    experiment_type='Preprocessed',
    accuracy=results_prep['accuracy'],
    precision=results_prep['precision'],
    recall=results_prep['recall'],
    f1=results_prep['f1'],
    loss=results_prep['loss']
)

print("✓ Experiment 1 completed and results logged")

---
## 8. Experiment 2: Raw Dataset
Train on raw public data, test on raw self-collected data

In [ ]:
print("\n" + "#"*60)
print("# EXPERIMENT 2: RAW DATASET")
print("#"*60 + "\n")

# Create data generators for raw data
train_gen_raw, _, test_gen_raw = create_data_generators(
    RAW_PUBLIC,
    None,  # No separate validation, will use validation_split
    RAW_SELF,
    use_augmentation=True
)

# Create validation generator from training data
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_gen_raw = val_datagen.flow_from_directory(
    RAW_PUBLIC,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    classes=CLASSES,
    subset='training'
)

val_gen_raw = val_datagen.flow_from_directory(
    RAW_PUBLIC,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    classes=CLASSES,
    subset='validation'
)

# Build new model
model_raw = build_efficientnetb2_model()

# Train model
history_raw = train_model(
    model_raw,
    train_gen_raw,
    val_gen_raw,
    model_name='efficientnetb2_raw',
    epochs=30,
    fine_tune=True
)

# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
axes[0].plot(history_raw.history['accuracy'], label='Train Accuracy')
axes[0].plot(history_raw.history['val_accuracy'], label='Val Accuracy')
axes[0].set_title('Model Accuracy - Raw')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

# Loss
axes[1].plot(history_raw.history['loss'], label='Train Loss')
axes[1].plot(history_raw.history['val_loss'], label='Val Loss')
axes[1].set_title('Model Loss - Raw')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Evaluate on test set
results_raw = evaluate_model(model_raw, test_gen_raw, "EfficientNetB2 - Raw")

# Log results
log_result(
    model_name='EfficientNetB2',
    experiment_type='Raw',
    accuracy=results_raw['accuracy'],
    precision=results_raw['precision'],
    recall=results_raw['recall'],
    f1=results_raw['f1'],
    loss=results_raw['loss']
)

print("✓ Experiment 2 completed and results logged")

---
## 9. Comparison Summary
Compare results from both experiments

In [ ]:
# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'Experiment': ['Preprocessed', 'Raw'],
    'Accuracy': [results_prep['accuracy'], results_raw['accuracy']],
    'Precision': [results_prep['precision'], results_raw['precision']],
    'Recall': [results_prep['recall'], results_raw['recall']],
    'F1-Score': [results_prep['f1'], results_raw['f1']],
    'Loss': [results_prep['loss'], results_raw['loss']]
})

print("\n" + "="*80)
print("EFFICIENTNETB2 - FINAL COMPARISON")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80 + "\n")

# Visualize comparison
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width/2, 
               [results_prep['accuracy'], results_prep['precision'], 
                results_prep['recall'], results_prep['f1']], 
               width, label='Preprocessed')
bars2 = ax.bar(x + width/2, 
               [results_raw['accuracy'], results_raw['precision'], 
                results_raw['recall'], results_raw['f1']], 
               width, label='Raw')

ax.set_ylabel('Score')
ax.set_title('EfficientNetB2: Preprocessed vs Raw Dataset')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.set_ylim([0, 1.1])
ax.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom',
                    fontsize=9)

plt.tight_layout()
plt.show()

print("\n✓ All experiments completed successfully!")
print("✓ Models saved in 'models/' directory")
print("✓ Results logged to 'final_results.csv'")